In [3]:
import zipfile
import os
import glob
import shutil

Extract the zip file contents

In [4]:
data_zip = '/content/Brain_tumor.zip'
data_dir = '/content/data/'
zip_file = zipfile.ZipFile(data_zip,'r')
zip_file.extractall(data_dir)

Create the 'train' and 'val' folders

In [5]:
os.mkdir(os.path.join(data_dir,'train'))
os.mkdir(os.path.join(data_dir,'val'))

Create the class(yes / no) sub-folders in train and val folders

In [6]:
data_train = '/content/data/train'
data_train_no = os.path.join(data_train,'no')
os.mkdir(data_train_no)
data_train_yes = os.path.join(data_train,'yes')
os.mkdir(data_train_yes)
data_val = '/content/data/val'
data_val_no = os.path.join(data_val,'no')
os.mkdir(data_val_no)
data_val_yes = os.path.join(data_val,'yes')
os.mkdir(data_val_yes)

Split the dataset

In [7]:
split = 0.8

no_size = len(os.listdir('/content/data/brain_tumor_dataset/no'))
yes_size = len(os.listdir('/content/data/brain_tumor_dataset/yes'))

for i,img in enumerate(glob.glob("/content/data/brain_tumor_dataset/no/*")):
  if(i < no_size*split):
    print(i,img)
    shutil.move(img,data_train_no)
  else:
    shutil.move(img,data_val_no)

for i,img in enumerate(glob.glob("/content/data/brain_tumor_dataset/yes/*")):
  if(i < yes_size*split):
    print(i,img)
    shutil.move(img,data_train_yes)
  else:
    shutil.move(img,data_val_yes)

0 /content/data/brain_tumor_dataset/no/no 100.jpg
1 /content/data/brain_tumor_dataset/no/no 6.jpg
2 /content/data/brain_tumor_dataset/no/6 no.jpg
3 /content/data/brain_tumor_dataset/no/22 no.jpg
4 /content/data/brain_tumor_dataset/no/25 no.jpg
5 /content/data/brain_tumor_dataset/no/34 no.jpg
6 /content/data/brain_tumor_dataset/no/18 no.jpg
7 /content/data/brain_tumor_dataset/no/no 923.jpg
8 /content/data/brain_tumor_dataset/no/no 89.jpg
9 /content/data/brain_tumor_dataset/no/no 10.jpg
10 /content/data/brain_tumor_dataset/no/No12.jpg
11 /content/data/brain_tumor_dataset/no/47 no.jpg
12 /content/data/brain_tumor_dataset/no/8 no.jpg
13 /content/data/brain_tumor_dataset/no/No14.jpg
14 /content/data/brain_tumor_dataset/no/44no.jpg
15 /content/data/brain_tumor_dataset/no/no.jpg
16 /content/data/brain_tumor_dataset/no/N17.jpg
17 /content/data/brain_tumor_dataset/no/30 no.jpg
18 /content/data/brain_tumor_dataset/no/no 8.jpg
19 /content/data/brain_tumor_dataset/no/no 90.jpg
20 /content/data/bra

In [8]:
import torch
import torchvision
from torchvision import datasets,transforms, models

Transformations of the train and val data

In [34]:
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(30),
                                       transforms.ToTensor(),
                                       torchvision.transforms.Normalize(
                                           mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225],),
                                       ])
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(30),
                                      transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],),
                                      ])

In [35]:
train_data = datasets.ImageFolder(data_train,transform = train_transforms)
test_data = datasets.ImageFolder(data_val,transform = test_transforms)

train_loader = torch.utils.data.DataLoader(train_data,shuffle=True,batch_size =16)
test_loader = torch.utils.data.DataLoader(test_data,shuffle=True,batch_size =16)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [12]:
import torch.nn as nn
import torch.nn.functional as F

1. Download the ResNet-18 pre-trained on ImageNet dataset
2. Freeze the model paramaters
3. Modify the last fully connected layer to train on the Brain Tumor dataset

In [36]:
model1 = models.resnet18(weights='IMAGENET1K_V1')
for par in model1.parameters():
  par.requires_grad = False

fc_feat = model1.fc.in_features
model1.fc = nn.Linear(fc_feat,2)

model1 = model1.to(device)

In [37]:
import torch.optim as optim
from torch.optim import lr_scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

Train and Test the model

In [38]:
best_accu = 0.0
for epoch in range(50):  # loop over the dataset multiple times
    print("Epoch",epoch)
    train_loss = 0.0
    val_loss = 0.0
    correct = 0
    model1.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        #print(inputs.size(0))
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #print("loss",loss)
        # print statistics
        train_loss += loss.item()*(inputs.size(0))
    train_loss = train_loss/len(train_loader.dataset)
    print("train_loss",train_loss)
    #print('Finished Training')


    model1.eval()
    for i, data1 in enumerate(test_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data1
        #print(inputs.size(0))
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()*(inputs.size(0))
        _, pred = torch.max(outputs, 1)
        correct += (pred == labels).sum()
    val_loss = val_loss/len(test_loader.dataset)
    print("val_loss",val_loss)
    acc = correct/len(test_loader.dataset)
    print('acc',acc.item()*100)


    if acc > best_accu:
      best_accu = acc
      print(f'Epoch:{epoch} Validation Accuracy:{acc:.4f}')
      torch.save(model1.state_dict(),'/content/data/model1_weights.pt')

Epoch 0
train_loss 0.843163135016493
val_loss 0.6108142280578613
acc 63.999998569488525
Epoch:0 Validation Accuracy:0.6400
Epoch 1
train_loss 0.6227585663055552
val_loss 0.5642387986183166
acc 68.00000071525574
Epoch:1 Validation Accuracy:0.6800
Epoch 2
train_loss 0.5089847771404999
val_loss 0.5302653932571411
acc 71.99999690055847
Epoch:2 Validation Accuracy:0.7200
Epoch 3
train_loss 0.46812543918933774
val_loss 0.4572082734107971
acc 75.99999904632568
Epoch:3 Validation Accuracy:0.7600
Epoch 4
train_loss 0.4293441842929483
val_loss 0.42784019231796266
acc 81.99999928474426
Epoch:4 Validation Accuracy:0.8200
Epoch 5
train_loss 0.3688948299790838
val_loss 0.39796730518341067
acc 79.99999523162842
Epoch 6
train_loss 0.3376528447484735
val_loss 0.3656394171714783
acc 85.999995470047
Epoch:6 Validation Accuracy:0.8600
Epoch 7
train_loss 0.3080551448126732
val_loss 0.36115365982055664
acc 87.99999952316284
Epoch:7 Validation Accuracy:0.8800
Epoch 8
train_loss 0.3197653239290115
val_loss 0.

In [39]:
model1.load_state_dict(torch.load('/content/data/model1_weights.pt'))
model1.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Test the trained model on sample image

In [45]:
from PIL import Image
test_image1 = '/content/data/val/yes/Y101.jpg'
test_image = '/content/data/val/no/11 no.jpg'
test_img = Image.open(test_image)
print(test_img)
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),  # adjust to model's expected size
    transforms.ToTensor(),
])
img_test = transform(test_img).unsqueeze(0)  # add batch dimension
img_test = img_test.to(device)
# Step 4: Run inference
with torch.no_grad():
    output = model1(img_test)
    print(output)
    _, prediction = torch.max(output,1)
    print(prediction)

print(f'Predicted class: {int(prediction)}')  # 0 or 1

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x168 at 0x7CA25A75FFD0>
tensor([[ 0.4182, -0.6014]], device='cuda:0')
tensor([0], device='cuda:0')
Predicted class: 0
